In [28]:
# Copyright 2022 The Forta Foundation
 
import requests
import pandas as pd
import json
import dotenv
import os

# Load environment variables
dotenv.load_dotenv()

True

This notebook allows to backtest the attack detector V2 against a set of alerts. Independent of the threshold set within the bot, the log will be analyzed, so the relevant alerts, their anomaly score, overall anomaly score, and any FP filtering mitigations are shown.

Since there is a command line character limit, the local persistence mechanism needs to be enabled to persist state across executions (by setting local node property in constants.py to 1)

In [29]:
# chain = "ethereum"
# attacks = [#("0x14d8Ada7A0BA91f59Dc0Cb97C8F44F1d177c2195","ethereum","tominu","2023-01-18","2023-01-27"),
#            #("0x69f0EdC352eCffC4EF49516c9b20eA88B3E947cb,0xceed34f03a3e607cc04c2d0441c7386b190d7cf4","ethereum","blueclues","2023-01-19","2023-01-26"),
#            #("0x88a2386e7ec97ad1e7a72176a66b6d0711ae3527","ethereum","quaternion","2023-01-17","2023-01-19"),
#            #("0x8a2d94ea342cbdd6d57db614b24f20cae286cac6,0x22898dc59145eae79471dd1f06b7e542e0982d30","ethereum","Upswing","2022-12-21","2023-01-20"),
#            #("0x15d87DC2Eb27fdA26451f8FB04C576639104344d","ethereum","roefinance","2023-01-10","2023-01-18"),
#            #("0x9D0163e76BbCf776001E639d65F573949a53AB03","ethereum","lendhub","2023-01-10","2023-01-16"),
#            #("0xc578d755cd56255d3ff6e92e1b6371ba945e3984","binance","ufdaoattack","2023-01-01","2023-02-06")
#            ("0x5a113147A564B4c5e17A274Bfa99F3Cc19E4843f".lower(),"ethereum","unshETH1","2023-05-24","2023-06-03"),
#            ("0x2525c811ecf22fc5fcde03c67112d34e97da6079","ethereum","Cellframe","2023-05-17","2023-06-01"),
#            ("0x0e816b0d0a66252c72af822d3e0773a2676f3278","ethereum","ERC20Token","2023-05-16","2023-06-08"),
#            ("0x1e8419e724d51e87f78e222d935fbbdeb631a08b","ethereum","Sturdy","2023-06-11","2023-06-20"),
#            ("0x24ae929ebe12589c5c193af5d7b03da05f5bfb7c","ethereum","Keep3r1","2023-05-30","2023-06-14"),
#            ("0x277b86a6adc925993419715278c8e318d8e27b85","ethereum","Keep3r2","2023-05-30","2023-06-14"),
#            ("0x91fa0770f46b0608a1f5273568621ddfe944c14f","ethereum","Keep3r3","2023-05-30","2023-06-14"),
#            ("0xbdf38b7475ff810325aa39e988fb80e0aa007e84","ethereum","Hashflow1","2023-05-30","2023-06-15"),
#            ("0xddb19a1bd22c53dac894ee4e2fbfdb0a06769216","ethereum","Hashflow2","2023-05-30","2023-06-25"),
#            ("0x7021c1b142eb634fa0749cda270c7aff74dc3b7f","ethereum","DEPUSDT","2023-06-06","2023-06-15"),
#            ("0x8f7370d5d461559f24b83ba675b4c7e2fdb514cc","ethereum","Pawnfi","2023-06-10","2023-06-18"),
#            
#            ]

chain = "binance"
attacks = [#("0x66be80c796cba0844dace3e291632bfd397bd7a0","binance","phyproxy","2023-01-01","2023-01-24"),
           #("0x1ae2dc57399b2f4597366c5bf4fe39859c006f99","binance","thoreum","2023-01-14","2023-01-20"),
           #("0x9BbD94506398a1459F0Cd3B2638512627390255e,0xda5919bf3a49ad47b7c7103a9ed3902cee78d528","binance","omniprotocol","2023-01-07","2023-01-18"),
           #("0xc578d755cd56255d3ff6e92e1b6371ba945e3984","binance","ufdaoattack","2023-01-01","2023-02-06"),
           #("0x67a909f2953fb1138beA4B60894B51291D2d0795,0xE2Ba15be8C6Fb0d7C1F7bEA9106eb8232248FB8B","binance","bratoken","2022-12-08","2023-01-16"),
           #("0xcf2362b46669e04b16d0780cf9b6e61c82de36a7","binance","GDS","2023-01-01","2023-01-05"),
           ("0xa100c4b1962dcdb44201ce123833bf275e5f8847","binance","DDExploit1","2023-05-31","2023-06-24"),
           ("0x0a3fee894eb8fcb6f84460d5828d71be50612762","binance","DDExploit2","2023-05-26","2023-06-01"),
           ("0xa247e81645d6b7fc800afecd836fef16f504d908","binance","unCompromise1","2023-06-05","2023-06-07"),
           ("0xf84efa8a9f7e68855cf17eaac9c2f97a9d131366","binance","unCompromise2","2023-05-05","2023-06-25"),
           ("0x8887a9387ee149a81a17145065a7b13132f732cd","binance","MuratiAI","2023-05-22","2023-06-16"),
           ("0xeade071ff23bcef312dec938ece29f7da62cf45b","binance","AtlantisLoan","2023-04-10","2023-06-26"),
           ("0x4beb05bf9bd4585ae2f4feb74112f18b423abb84","binance","TrustTheTrident1","2023-06-10","2023-06-13"),
           ("0x0060129430df7ea188be3d8818404a2d40896089","binance","TrustTheTrident2","2023-05-31","2023-06-13"),
           ("0x106016cdc3878c7ff2b386303000db0001d838eb","binance","CFC","2023-06-11","2023-06-16"),
           ("0xf84efa8a9f7e68855cf17eaac9c2f97a9d131366","binance","Ara","2023-05-05","2023-06-26"),
           ("0x4b92cc3452ef1e37528470495b86d3f976470734","ethereum","Midas","2023-06-15","2023-06-19"),
            ]

In [30]:
# #clustering
# for attacker_address in attacker_addresses.split(","):
#     sql = f"SELECT metadata FROM forta.{chain}_alerts WHERE bot_id = '0xd3061db4662d5b3406b52b20f34234e462d2c275b99414d76dc644e2486be3e9' and CAST(substring(block_timestamp,1,19) as datetime)  >= '2022-10-01T00:00:00' AND CAST(substring(block_timestamp,1,19)  as datetime)  <= '{end_date}T00:00:00' and  lower(arrayStringConcat(metadata)) like '%{attacker_address.lower()}%'"
#     print(sql)
#     df = execute_query(sql)
#     print(df)

In [31]:
def get_alert_hashes(attacker_addresses: str, start_date: str, end_date: str) -> pd.DataFrame:

    alert_hashes_df = pd.DataFrame(columns=["hash","createdAt"])
    for attacker_address in attacker_addresses.split(","):
        url = 'https://api.forta.network/graphql'
        query = '''query exampleQuery {
            # first 5 alerts
            alerts(
                input: { first: 2000 , addresses: ["'''+attacker_address+'''"],
                blockDateRange: { startDate: "'''+start_date+'''", endDate: "'''+end_date+'''" }
                }
            ) {
                pageInfo {
                hasNextPage
                endCursor {
                    alertId
                    blockNumber
                }
                }
                alerts {
                hash
                createdAt
                
                }
            }
            }
        '''
        r = requests.post(url, json={'query': query})
        #print(r.text)
        json_data = json.loads(r.text)
        df = pd.DataFrame(json_data['data']['alerts']['alerts'])
        #print(df)
        alert_hashes_df = pd.concat([alert_hashes_df, df])

    
    return alert_hashes_df


In [32]:

#write to file run.sh
with open("run_binance.sh", "w") as f:

    for attacker_addresses, chain, name, start_date, end_date in attacks:

        alert_hashes_df = get_alert_hashes(attacker_addresses.lower(), start_date, end_date)
        print(f"Got {len(alert_hashes_df)} alerts.")
        f.write(f"echo 'output.txt' > {name}_{chain}.txt\n")

        #run npm run to get and route to output.txt
        count = 0
        alert_hashes = ""
        for alert_hash in alert_hashes_df["hash"]:
            f.write(f"echo {name}: {alert_hash}\n")
            f.write(f"npm run alert {alert_hash} >> {name}_{chain}.txt 2>> {name}_{chain}_error.txt\n")
            #!npm run alert {alert_hash} >> {name}.txt 2>> {name}_{chain}.txt


        f.write(f"grep 'ERROR'  {name}_{chain}.txt >  {name}_{chain}_debug.txt\n")
        f.write(f"grep 'WARN'  {name}_{chain}.txt >>  {name}_{chain}_debug.txt\n")
        f.write(f"grep 'adding cluster mapping'  {name}_{chain}.txt >>  {name}_{chain}_debug.txt\n")
        f.write(f"grep 'adding FP mitigation cluster'  {name}_{chain}.txt >>  {name}_{chain}_debug.txt\n")
        f.write(f"grep 'anomaly'  {name}_{chain}.txt >>  {name}_{chain}_debug.txt\n")
        f.write(f"grep 'likely involved in an attack'  {name}_{chain}.txt >>  {name}_{chain}_results.txt\n")
        f.write(f"grep 'ATTACK-DETECTOR'  {name}_{chain}.txt >>  {name}_{chain}_results.txt\n")



Got 395 alerts.
Got 75 alerts.
Got 5 alerts.
Got 831 alerts.
Got 65 alerts.
Got 1306 alerts.
Got 18 alerts.
Got 136 alerts.
Got 106 alerts.
Got 841 alerts.
Got 241 alerts.
